# 1 | Understand

# 1.1 | Read OpenML


In [ ]:
data_id = 43406
import pandas as pd
from sklearn.datasets import fetch_openml
df = fetch_openml(parser='auto', data_id=data_id)

In [ ]:
data = df.data.copy()
target = df.target()

# 1.2 | Read CSV, XLSX

In [ ]:
filename = ''
import pandas as pd
df = pd.read_csv(filename, header=0)
# ---- EXCEL
import pandas as pd
df = pd.read_excel(filename) 

In [ ]:
label = 'class'
features = [n for n in df.columns if n != label]

target = data[label].copy()
data = df[features].copy()
data.drop(label,  axis=1, inplace=True)

# 1.3 | EDA - Pandas

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
data.describe(include=object).T

In [ ]:
data.corr()

In [ ]:
col_name = ''
data.groupby(col_name).count()

In [ ]:
target.value_counts()

In [ ]:
# _ = data.hist(figsize=(15,15))
# _ = data.plot.scatter(x='length', y='width', c='DarkBlue')
# _ = pd.scatter_matrix(data[features])
# _ = pd.plotting.scatter_matrix(data, alpha=0.2, figsize=[15,15], c='darkblue')
# _ = data.boxplot(column=['age', 'pclass'], figsize=[10,10])

# 1.4 | EDA - Pandas Profiling

In [ ]:
try:
  from pandas_profiling import ProfileReport 
except:
  !pip install -q pandas-profiling
  from pandas_profiling import ProfileReport 

In [ ]:
report = ProfileReport(data)
report

In [ ]:
#  report.to_file("report.html")

# 1.5 | EDA - SweetViz

In [ ]:
try:
  from pandas_profiling import ProfileReport 
except:
  !pip install -q sweetviz
  from pandas_profiling import ProfileReport 

In [ ]:
report = sv.analyze(data)
report.show_notebook()

In [ ]:
# comparison_report = sv.compare([data_train,'Train'], [data_test,'Test'], target_feat='class')
# comparison_report.show_notebook()

In [ ]:
# report.show_html('report.html')

# 1.6 | EDA - DataPrep

In [ ]:
try:
    from dataprep.eda import create_report, plot, plot_correlation, plot_missing, plot_diff
    from dataprep.clean.clean_headers import clean_headers  
except:
    !pip install -q dataprep
    from dataprep.eda import create_report, plot, plot_correlation, plot_missing, plot_diff
    from dataprep.clean.clean_headers import clean_headers 

In [ ]:
create_report(data)

In [ ]:
# clean_headers(data, case='camel', replace={'XXX': 'YYY'})     # Cleaning the header-names / feature-names 
# plot(data)                  # Histogram for all feartures
# plot_missing(data)          # missing data
# plot_missing(data, "...")   # Impact of missing data
# plot_correlation(data)      # correlation
# plot(data, "...", "...")    # Relationship between two features
# plot(data, "...")           # Single feature

# 1.7 | EDA - Plotly

In [ ]:
import plotly.express as px

In [ ]:
# Boxplot
title_ = 'Titel-Text'
px.box(data[['col1', 'col2']], title=title_, width=600, height=600)

In [ ]:
# Scattergramm
title_ = 'Titel-Text'
px.scatter(data, x='col1', y='col2', color='target', labels={"col1": "xxxx","col2": "yyyy"}, title=title_, width=1200, height=600)

In [ ]:
# Scatter Matrix
px.scatter_matrix(data, dimensions=data.columns, opacity=1)
# px.scatter_matrix(data, dimensions=data.columns, opacity=1, color=target)

In [ ]:
# Linie
px.line(data, x= 'col1', y='col2', width=800, height=400)

# 1.8 | EDA - PyGWalker

In [ ]:
try:
  import pygwalker as pyg
except:
  !pip install -q pygwalker
  import pygwalker as pyg

In [ ]:
pyg.walk(data)

# 2 | Prepare

# 2.1 | Datentyp ermitteln




In [ ]:
all_col = data.columns
num_col = data.select_dtypes(include='number').columns
cat_col = data.select_dtypes(exclude='number').columns

# 2.2 | Datentyp ändern

In [ ]:
df['column_name'] = df['column_name'].astype(int)
df['column_name'] = df['column_name'].astype(float)
df['column_name'] = df['column_name'].astype(bool)

# 2.3 | Löschen von Spalten

In [ ]:
# Löschen von Spalten
data.drop(['col1', 'col2'], axis=1, inplace=True)

# 2.4 | Berechnen von Spalten

In [ ]:
data['col3'] = data['col1'] + data['col2']

# 2.5 | Spalten neu benennen

In [ ]:
data.rename(columns = {'old_col1':'new_col1', 'old_col2':'new_col2'}, inplace = True)
data.columns = ['new1', 'new2', 'new3', 'new4']
data.columns = data.columns.str.replace('old_char', 'new_char')

# 2.6 | Missing Values

# 2.6.1 | Prüfen auf Missing Values

In [ ]:
# --- Berechne die Anzahl der fehlenden Werte pro Spalte
data.isna().sum()

# 2.6.2 | Python Standard

In [ ]:
# --- Entferne Zeilen mit fehlenden Daten
data.dropna(inplace=True)
# --- Ersetze durch Mittelwert, Median, Mode
data['A'].replace([numpy.nan], data['A'].mean(), inplace=True)
data['B'].replace([numpy.nan], data['B'].median(), inplace=True)
data['C'].replace([numpy.nan], data['C'].mode()[0], inplace=True)
# --- Ersetze durch Vorgänger/Nachfolger
data.fillna(method='ffill', inplace=True)
data.fillna(method='bfill', inplace=True)
# --- Ersetze durch Konstante -999
data.fillna(value=-999, inplace=True)


# 2.6.3 | SciKit-Learn

In [ ]:
# --- sklearn
from sklearn.impute import SimpleImputer
data = SimpleImputer(missing_values=pd.nan, strategy='most_frequent').fit_transform(data[cat_col])

# 2.7 | Outlier

# 2.7.1 | Outlier via Standardabweichung
Mehr als 3 Standardabweichungen vom Mittelwert entfernt

In [ ]:
# Entferne Datenpunkte, die mehr als 3 Standardabweichungen vom Mittelwert entfernt sind
data = data[(data - data.mean()) / data.std() < 3].dropna()

# 2.7.2 |  Outlier via 1.5 IQR (Boxplot)


In [ ]:
# calculate the first and third quartiles (Q1 and Q3)
Q1 = data['col1'].quantile(0.25)
Q3 = data['col1'].quantile(0.75)
# calculate the interquartile range (IQR)
IQR = Q3 - Q1
# set the lower and upper bounds for outlier detection
lower_bound = Q1 - (1.5 * IQR)
upper_bound = Q3 + (1.5 * IQR)
# identify outliers using the bounds
outliers = (data['col1'] < lower_bound) | (data['col1'] > upper_bound)
# set outliers to NA
data_no_outliers = data[~outliers]
# remove rows with NA
data_no_outliers.dropna(inplace=True)

# 2.7.3 | Outlier via Winsorizing 
Begrenzung min/max auf Grenze 5% und 95% Werte

In [ ]:
from scipy.stats.mstats import winsorize
# Begrenzung der Gültigkeit auf 5% - 95%, Anzahl bleibt erhalten, Werte <5% bzw. >95% werden auf Unter-/Obergrenze gesetzt
data_winsorized = pd.DataFrame(winsorize(data, limits=[0.05, 0.05]), columns=data.columns)

# 2.7.4 | Outlier via LOF

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(n_neighbors=5) # Ausreisser, wenn er sich stark von seinen Nachbarn unterscheidet
outliers = lof.fit_predict(pd.DataFrame(target))
# outliers = lof.fit_predict(data['colname'])
num_outliers = len(outliers[outliers == -1])
print("Anzahl der Ausreißer:", num_outliers)

# 2.8 | Codierung & Skalierung

# 2.8.1 | OrdinalEncoder (ordinal)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
data[cat_col] = OrdinalEncoder().fit_transform(data[cat_col])

In [ ]:
data.col.replace(['yes', 'no'], [1, 0], inplace = True)

# 2.8.2 | OneHotEncoder (nominal)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(dtype=int, sparse_output=False, handle_unknown='ignore').fit_transform(data[cat_col])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(dtype=int, drop='if_binary', handle_unknown='ignore').fit_transform(data[cat_col])

# 2.8.3 | LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder
target = LabelEncoder().fit_transform(target)
# target.replace(['yes', 'no'], [1, 0], inplace = True)

# 2.8.4 | Normalisierung

In [ ]:
from sklearn.preprocessing import MinMaxScaler
data[num_col] = MinMaxScaler().fit_transform(data[num_col])

# 2.8.5 | Standardisierung

In [ ]:
from sklearn.preprocessing import StandardScaler
data[num_col] = StandardScaler().fit_transform(data[num_col])

# 2.9 | Imbalanced 

# 2.9.1 | Upsampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
data_resampled, target_resampled = ros.fit_resample(data, target)

# 2.9.2 | Downsampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
data_resampled, target_resampled = rus.fit_resample(data, target)

# 2.9.3 | Klassen gewichten

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(target_train), target_train)
model.fit(data_train, data_train, class_weight=class_weights)


# 2.9.4 | SMOTE   
(Synthetic Minority Over-sampling Technique) 

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
data_resampled, target_resampled = smote.fit_resample(data, target)

# 2.9.5 | ADASYN   
(Adaptive Synthetic Sampling)

In [ ]:
from imblearn.over_sampling import ADASYN
ada = ADASYN()
data_resampled, target_resampled = ada.fit_resample(data, target)

# 3 | Modeling
---

# 3.1 | Train-Test-Split

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.3, random_state=42, stratify=target)
data_train.shape, data_test.shape, target_train.shape, target_test.shape

# 3.2.| Train-Val-Test-Split

[train-val-test](https://towardsdatascience.com/how-to-split-data-into-three-sets-train-validation-and-test-and-why-e50d22d3e54c)

In [ ]:
from fast_ml.model_development import train_valid_test_split
data_train, target_train, data_valid, target_valid, data_test, target_test = train_valid_test_split(data, target)                                                 

# 3.3 | Modelle Klassifikation

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

model = estimator()
model.fit(data, target)

# 3.4 | Modelle Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

model = estimator()
model.fit(data, target)

# 3.5 | Modelle MultiLayerPerceptron

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

model = estimator()
model.fit(data, target)

# 3.6 | Modelle Association

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder

model = estimator()
model.fit(data, target)

# 3.6 | Modelle Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering

model = estimator()
model.fit(data, target)

# 3.7 | Modelle Anomaly/Deviation

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

from pyod.models.knn import KNN
from pyod.models.lof import LOF

model = estimator()
model.fit(data, target)

# 3.8 | Modelle Ensemble

# 3.8.1 | Stacking Classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

In [ ]:
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('knn', KNeighborsClassifier()),
    ('svc', make_pipeline(StandardScaler(), LinearSVC(random_state=42)))]

In [ ]:
model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression()) 
model.fit(data_train, target_train)

# 3.8.2 | Stacking Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.ensemble import StackingRegressor

In [ ]:
estimators = [
    ('rf', RandomForestRegressor(n_estimators=50, max_features=5, max_depth=20, min_samples_leaf=5, random_state=42)),
    ('knn', KNeighborsRegressor(4)),
    ('lreg', LinearRegression())]

In [ ]:
model = StackingRegressor(estimators=estimators, final_estimator=RidgeCV())
model.fit(data_train, target_train) 

# 4 | Evaluate
---

# 4.1 | Evaluate Klassifikation

# 4.1.1 | Prognose

In [ ]:
target_train_pred = model.predict(data_train)
target_test_pred = model.predict(data_test)

In [ ]:
# Prognose 2 neue Datensätze an das Modell übergeben
# new_data = { 'feat1': [1, 2], 'feat2': [0, 1], 'feat4': [33, 80], 'feat5': [0, 0], 'feat6': [0, 0] }
# new = pd.DataFrame(new_data)
# model.predict(new)

# 4.1.2 | Accuracy

In [ ]:
from sklearn.metrics import accuracy_score
acc_train = accuracy_score(target_train, target_train_pred) * 100
print (f"Modell: {model} -- Train -- Accuracy: {acc_train:5.2f}")

In [ ]:
acc_test = accuracy_score(target_test, target_test_pred) * 100
print (f"Modell: {model} -- Test -- Accuracy: {acc_test:5.2f}")

# 4.1.3 | Precision, Recall, F1-Score

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(target_test, target_test_pred)
recall = recall_score(target_test, target_test_pred)
f1 = f1_score(target_test, target_test_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# 4.1.4 | Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
conf_matrix = confusion_matrix(target_test, target_test_pred)
display_labels_=['No','Yes']
disp = ConfusionMatrixDisplay(conf_matrix, display_labels=display_labels_) 
disp.plot(cmap='Blues')

In [ ]:
print(classification_report(target_test, target_test_pred, target_names=display_labels_))

# 4.1.5 | Cohen's Kappa

In [ ]:
from sklearn.metrics import cohen_kappa_score
target_pred = model.predict(data_train)
cks_train = cohen_kappa_score(target_train, target_pred)
print (f"Modell: {model} -- Train -- Cohen's Kappa: {cks_train:5.2f}")

# 4.1.6 | ROC/AUC - sklearn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

target_test_probabilities = model.predict_proba(data_test) 

fpr, tpr, thresholds = roc_curve(target_test, target_test_probabilities)
auc = roc_auc_score(target_test, target_test_probabilities)

# Plot the ROC curve
plt.plot(fpr, tpr, label='ROC Curve (AUC = {:.2f})'.format(auc))
plt.plot([0, 1], [0, 1], linestyle='--', color='r', label='Random')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

# 4.1.7 | ROC/AUC - yellowbrick

In [ ]:
from yellowbrick.classifier import ROCAUC
classes_ = ['class1','class2']
visualizer = ROCAUC(model, classes=classes_)

In [ ]:
visualizer.fit(data_train, target_train)        
visualizer.score(data_test, target_test)       
visualizer.show()      

# 4.1.8 | TPR & FPR

In [ ]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(target_test, target_test_pred).ravel()
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print("True Positive Rate (TPR):", tpr)
print("False Positive Rate (FPR):", fpr)


# 4.2. | Evaluate Regression

# 4.2.1 | Prognose

In [ ]:
target_train_pred = model.predict(data_train)
target_test_pred = model.predict(data_test)

In [ ]:
# Prognose 2 neue Datensätze an das Modell übergeben
# new_data = { 'feat1': [1, 2], 'feat2': [0, 1], 'feat4': [33, 80], 'feat5': [0, 0], 'feat6': [0, 0] }
# new = pd.DataFrame(new_data)
# model.predict(new)

# 4.2.2 | Bestimmtheitsmass - r2

In [ ]:
from sklearn.metrics import r2_score
r2_train = r2_score(target_train, target_train_pred)
print(f'Modell: {model}\n -- Train --- Bestimmtheitsmass: {r2_train:5.2f}' )

In [ ]:
r2_test = r2_score(target_test, target_test_pred)
print(f'Modell: {model}\n -- Test --- Bestimmtheitsmass: {r2_test:5.2f}' )

# 4.2.3 | Mean Absolut Error - MAE

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(target, target_test_pred)
print(f'Modell: {model} -- Test -- Mean Absolute Error: {mae:5.2f}' )

# 4.2.4 | Regressionskoeffizienten

In [ ]:
print('Intercept/Ursprung: %10.2f' %float(model.intercept_))
anzahl = len(model.coef_)
print(f'Slope/Steigung der {anzahl} Merkmale: \n ', model.coef_)

# 4.3 | Evaluate Clustering

# 4.3.1 | Silhouette Koeffizient 

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_coef = silhouette_score(data, model.labels_)
print("Silhouette-Koeffizient:", silhouette_coef)

# 4.3.2 |  Calinski-Harabasz-Index

In [ ]:
from sklearn.metrics import calinski_harabasz_score
ch_score = calinski_harabasz_score(data, model.labels_)
print("Calinski-Harabasz Index:", ch_score)


# 4.3.3 | Rand Index

In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score
rand_score = adjusted_rand_score(target, model.labels_)
print("Rand Index:", rand_score)

# 4.3.4 | Homogenität & Vollständigkeit

In [ ]:
from sklearn.metrics import homogeneity_score, completeness_score
homogeneity = homogeneity_score(target, target_pred)
completeness = completeness_score(target, target_pred)
print("Homogenität:", homogeneity)
print("Vollständigkeit:", completeness)

# 4.4 | Feature Importance

# 4.4.1 | Ermittlung & Visualisierung

In [ ]:
from yellowbrick.model_selection import FeatureImportances
viz = FeatureImportances(model)
viz.fit(data, target)
viz.show()

# 4.4.2 | Visualisierung Modellattribut

In [ ]:
import plotly.express as px
px.bar(x=model.feature_importances_, y=data.columns).update_yaxes(categoryorder="total ascending")

#4.5 | Feature Selection

# 4.5.1 | Select Best

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
feature_selection = SelectKBest(score_func=f_regression, k=6)

# 4.5.2 | Recursive Feature Elimination


In [ ]:
%matplotlib inline
from yellowbrick.model_selection import RFECV
from sklearn.model_selection import KFold

In [ ]:
cv = KFold(n_splits=2)
visualizer = RFECV(model, cv=cv, scoring='accuracy')
visualizer.fit(data_train, target_train)       
visualizer.show()  

In [ ]:
visualizer.support_, visualizer.ranking_

# 4.5.3 | Dimension Reduction - PCA
Principal Component Analysis 

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
data_new = pca.fit_transform(data)

# 4.5.4 | Dimension Reduction - LDA
Linear Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=2)
data_new = lda.fit_transform(data, target)

# 4.6 | Cross Validation

In [ ]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(model, data_train, target_train, 
                            scoring='r2', cv=cv, return_train_score=True, return_estimator=True)

In [ ]:
cv_results

In [ ]:
train_result = cv_results['test_score'].mean()
val_result = cv_results['train_score'].mean()
print(f"Train {train_result:.2f} -- Validation {val_result:.2f}")

# 4.7 | HyperparameterTuning

# 4.7.1 | GridSearch

In [ ]:
param_grid = {}
param_grid['param1'] = [10, 50, (10,10,10)]
param_grid['param2'] = ['identity','logistic','relu']
param_grid['param3'] =['lbfgs','sgd','adam']
param_grid['param4'] = ['constant','invscaling','adaptive']

In [ ]:
from sklearn.model_selection import GridSearchCV
gs_cv = GridSearchCV(model, param_grid, cv=2, scoring=['accuracy'],return_train_score= True, verbose=1)

In [ ]:
gs_cv.fit(data_train, target_train)

In [ ]:
gs_cv.best_estimator_
gs_cv.best_params_
gs_cv.best_score_

In [ ]:
gs_cv.cv_results_

In [ ]:
l_mta = list()
l_params = list()

all_results = gs_cv.cv_results_
for mta, params in zip(all_results["mean_test_accuracy"], all_results['params']):
  l_mta.append(mta)
  l_params.append(params)

results = pd.DataFrame(columns=['MTA', 'Parameter'])
results.MTA = l_mta
results.Parameter = l_params
results

# 4.7.2 | RandomizedSearch

In [ ]:
import numpy as np

In [ ]:
param_grid = {}
param_grid['param1'] = np.arange(10, 100, 10)
param_grid['param2'] = np.arange(10, 30, 5)
param_grid['param3'] = np.arange(2, 10, 2)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs_cv = RandomizedSearchCV(estimator = model, param_distributions=param_grid, n_iter=15, cv=5, scoring="r2", verbose=2, random_state=42)
rs_cv.fit(data_test,target_test)

In [ ]:
rs_cv.best_estimator_
rs_cv.best_params_
rs_cv.best_score_

In [ ]:
rs_cv.cv_results_

In [ ]:
l_mts = list()
l_params = list()

all_results = rs_cv.cv_results_
for mts, params in zip(all_results["mean_test_score"], all_results['params']):
  l_mts.append(mts)
  l_params.append(params)

results = pd.DataFrame(columns=['MTS', 'Parameter'])
results.MTS = l_mts
results.Parameter = l_params
results.sort_values(by=['MTS'], ascending=False)

# 4.8 | Results Analysis

# 4.8.1 | Prepare Results

# 4.8.2 | Box-Plots

# 4.9 | Verschiedenes 

# 4.9.1 | Discrimination Threshold


In [ ]:
from yellowbrick.classifier import DiscriminationThreshold

# Instantiate the classification model and visualizer
visualizer = DiscriminationThreshold(model, n_trials=1, random_state=42)
visualizer.fit(data, target, random_state=42)        # Fit the data to the visualizer
visualizer.show()                   # Finalize and render the figure

# 4.9.2 | Round Prediction

In [ ]:
rounded_predictions = np.argmax(predictions, axis = -1)

# 5 Deploy
---

# 5.1 | Save Model (joblib)

In [ ]:
import joblib
joblib.dump(model, '/content/cv-model.pkl')

# 5.2 | Load Model (Joblib)

In [ ]:
import joblib
model_geladen = joblib.load('/content/cv-model.pkl')

# 5.3 | Save Model (pmml)

In [ ]:
%%capture
try:
    from sklearn2pmml import sklearn2pmml, make_pmml_pipeline
    from pypmml import Model
except:
    !pip install -q sklearn2pmml
    !pip install -q pypmml
    from sklearn2pmml import sklearn2pmml, make_pmml_pipeline
    from pypmml import Model

In [ ]:
pmml_pipe = make_pmml_pipeline(model, data.columns.values, target.name)
sklearn2pmml(pmml_pipe, "trained_model.pmml", with_repr = True)

# 5.4 | Load Model (pmml)

In [ ]:
from pypmml import Model
model_load = Model.load('/content/trained_model.pmml')

# 5.5 | Save Model (pickle)

In [ ]:
import pickle
pickle.dump(model, open(model_file_path, 'wb'))

# 5.6 | Load Model (pickle)

In [ ]:
model = pickle.load(open(model_file_path, 'rb'))

# 6 | Keras

# 6.1 | DataFrames in Arrays

In [ ]:
import numpy as np
data = np.array(data)
target = np.array(target)
data = np.asarray(data).astype(np.float32)
data.shape, target.shape

# 6.2 | Import Layer & Model

In [ ]:
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential

# 6.3 | Aufbau Neuronales Netz

# 6.3.1 | Klassifizierung

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=[15, ]))             # Input Layer 15 Parameter
model.add(Dense(45, activation="relu"))           
model.add(Dense(45, activation="relu"))           
model.add(Dense(2, activation="softmax"))      

In [ ]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(data_train, target_train, validation_split=0.3, epochs=15)
print(history.params)
print(history.history.keys())

# 6.3.2. | Regression

In [ ]:
model = Sequential()
model.add(Dense(units=13, input_dim=13, kernel_initializer='normal', activation='relu'))          # 13 Input Paramter  
model.add(Dense(units=600, kernel_initializer='normal', activation='relu'))                          
model.add(Dense(units=600, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=600, kernel_initializer='normal', activation='relu')) 
model.add(Dense(units=600, kernel_initializer='normal', activation='relu'))             
model.add(Dense(units=600, kernel_initializer='normal', activation='relu'))             
model.add(Dense(units=8, kernel_initializer='normal', activation='relu'))            
model.add(Dense(1, kernel_initializer='normal')) 

In [ ]:
from tensorflow import keras
import tensorflow as tf
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse', optimizer=opt, metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
history = model.fit(data_train, target_train, epochs=100, validation_split=0.3)
print(history.params)
print(history.history.keys())

# 6.4 | Anzeigen Netzwerk

In [ ]:
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)